In [ ]:
### Try makingProgram to observe splitting of clusters
'''
1) Cluster our cells using GMM after UMAP projeciton
2) Find the optimal # of clusters for each slice using BIC
3) See when # of clusters increases
4) Perhaos take the means of these clusters at each slice and try to charectorize how they change
'''
from kneed import KneeLocator
import ubergauss as ug

tempData = [x.copy() for x in transformedData]
tempDataStacked = np.vstack(tempData)

maxClust = 32
nComponentsList=range(1,maxClust+1)

currentMax = maxClust
minBICs = [np.inf]*len(tempData)
minNumComponents = [np.nan]*len(tempData)
bestLabels = [None]*len(tempData)

save=True
saveName='Sim5BIC'
methodOfOptBIC = 'uber' #'uber' or 'kneed' or 'minAdjBIC'

titles = ['Time 0', 'Time 1', 'Time 2', 'Time 3', 'Time 4', 'Time 5']
for i in range(len(tempData)-1, -1, -1):
    print(i)
    d = tempData[i]
    #models = [GaussianMixture(n_components=n, covariance_type='full', random_state=1337).fit(tempDataStacked) for n in nComponentsList]
    models = [GaussianMixture(n_components=n, covariance_type='full', random_state=1337).fit(d) for n in nComponentsList]
    
    bic = [m.bic(d) for m in models]
    
    
    plt.plot(nComponentsList[:maxClust], bic[:maxClust], '-o')
    plt.title(f'{titles[i]}')
    plt.xlabel("Number of Components")
    plt.ylabel("BIC")
    plt.xticks([el for el in nComponentsList[:maxClust] if el%2==0], [el for el in nComponentsList[:maxClust] if el%2==0])
    plt.tight_layout()
    if save:
        plt.savefig(f'{saveName}_{i}.png')
    plt.show()

    if methodOfOptBIC=='kneed':
        kn = KneeLocator(nComponentsList, bic ,curve='convex', direction='decreasing')
        kn.plot_knee_normalized()
        bestNumClusters = kn.knee
        print(bestNumClusters)
        bestBICIndex = nComponentsList.index(bestNumClusters)
        print(bestBICIndex)
    elif methodOfOptBIC == 'uber':
        bestBICIndex = ug.diag_maxdist(bic[:maxClust])
        #plt.scatter(nComponentsList[:maxClust], bic[:maxClust])
        #plt.show()
        #maxClust = bestBICIndex+3
    elif methodOfOptBIC == 'minAdjBIC':
        #bic = [b + (np.log(n)*(len(tempData))) for b,n in zip(bic, nComponentsList)]
        bic = [b for b,n in zip(bic, nComponentsList)]
        bic=bic[:maxClust]
        print(bic)
        bestBICIndex = np.argmin(bic)
        print(bestBICIndex)
        maxClust = bestBICIndex+1
    print(f"Best num Clusters is {nComponentsList[bestBICIndex]}\n")
    print(models[bestBICIndex])
    bestLabels[i] = models[bestBICIndex].predict(d)
                
print("")
print("All Mins Found")

#bestLabels = list(reversed(bestLabels))
numToAdd=0
for i,l in enumerate(bestLabels):
    bestLabels[i] = np.asarray([x+numToAdd for x in bestLabels[i]])
    numToAdd = max(bestLabels[i])+1
draw.auto_tiny(tempData, bestLabels, wrap='test', legend=False, dim=(1,len(tempData)), same_limit=True)

for i,l in enumerate(bestLabels):
    if Labels is not None:
        print(f"ARI:{i} {ARI(Labels[i], l)}")
    

In [ ]:
### Preliminary realignment

def prelimAlign(d1, d2, Y2):
    ### Use hungarian to at least make main clusters have the
    ### same labels across time

    (r,c),distances,X=getHung(d1,d2)
    
    d1Keys = list(d1.keys())
    d2Keys = list(d2.keys())
    D1Clusters=[d1Keys[r[index]] for index in range(len(r))]
    D2Clusters=[d2Keys[c[index]] for index in range(len(c))]
    
    #D1Clusters=[]
    #D2Clusters=[]
    #for index in range(len(r)):
        #print(f"replacing: {d2Keys[c[index]]} with {d1Keys[r[index]]}")
        #D1Clusters.append(d1Keys[r[index]])
        #D2Clusters.append(d2Keys[c[index]])
    

    transformationDict = {k2:D1Clusters[i] for i, k2 in enumerate(D2Clusters)}
    #for i, k2 in enumerate(D2Clusters):
        #transformationDict[k2] = D1Clusters[i]
    for k2 in d2.keys():
        if k2 not in transformationDict.keys() and k2 not in D1Clusters:
            transformationDict[k2]=k2
        elif k2 not in transformationDict.keys() and k2 in D1Clusters:
            transformationDict[k2]=max(D2Clusters)+2
    print(transformationDict)

    #Actually replaces the values
    Y2 = np.asarray([transformationDict[y] for y in Y2])
    return Y2

earlyAlignMethod = 'cellsaw'
if earlyAlignMethod == 'cellsaw':
    from cellsaw.merge.draw import confuse
    for i in range(len(tempData)-1):
        y1, y2 = confuse(bestLabels[i].copy(), bestLabels[i+1].copy())
        print(y1)
        print(y2)
        bestLabels[i+1] = y2
        
elif earlyAlignMethod == 'prelim':
    for i in range(len(clusterCentres)-1):
        labels1 = bestLabels[i].copy()
        labels2 = bestLabels[i+1].copy()

        #labels2 = prelimAlign(clusterCentres[i], clusterCentres[i+1], labels2)
        bestLabels[i+1] = prelimAlign(clusterCentres[i], clusterCentres[i+1], bestLabels[i+1])

        #print(labels2)

In [ ]:
#from cellsaw.merge.draw import confuse
   
def basicMatch(labels1, labels2, data1, data2, hungmatch):

    ### This prevents some clusters from not being assigned
    ### to any clusters in the next time-slice
    threshold=0.5
    nonZeros=[0]
    while 0 in nonZeros and threshold>0:
        y1map, y2map, canvas = hungutil.make_canvas_and_spacemaps(labels1, labels2, hungmatch, normalize=True, dist = False)
        row_ind, col_ind = hungutil.solve_dense(canvas)
        canvas, canvasbackup = hungutil.clean_matrix(canvas, threshold=threshold)
        nonZeros=np.count_nonzero(canvas, axis=1)
        threshold-=0.01
    draw.doubleheatmap(canvasbackup, canvas, y1map, y2map, row_ind, col_ind)
    
    ### Get corresponding labels
    sorting = sorted(zip(col_ind, row_ind))
    col_ind, row_ind= list(zip(*sorting))

    # some rows are unused by the matching,but we still want to show them:
    order= list(row_ind) + list(set(y1map.integerlist)-set(row_ind) )
    canvas = canvas[order]
    x0labels = list(y2map.itemlist)
    y0labels = [y1map.getitem[r]for r in order]
    x,y=np.nonzero(-canvas)
    
    labelsMatch = {y0labels[x[i]]:[] for i in range(len(x))}
    for i in range(len(x)):
        labelsMatch[y0labels[x[i]]].append(x0labels[y[i]])
    draw.auto_tiny([data1, data2], [labels1, labels2], wrap='test', dim=(1,2), same_limit=True)
    
    return labelsMatch, canvas, (x0labels, y0labels)


labelsMatchList=[]
simMatrices=[]
xyLabs=[]
for i in range(len(tempData)-1):
    hungmatch, distances = util.hungarian(tempData[i], tempData[i+1])
    labelsMatch, canvas, xyLab = basicMatch(bestLabels[i].copy(), bestLabels[i+1].copy(), tempData[i].copy(), tempData[i+1].copy(), hungmatch)
    print(labelsMatch)
    labelsMatchList.append(labelsMatch)
    simMatrices.append(canvas)
    xyLabs.append(xyLab)



In [ ]:
from cellsaw.merge.draw import confuse
for i in range(len(tempData)-1):
    y1,y2 = confuse(bestLabels[i].copy(), bestLabels[i+1].copy())
    print(y1)
    print(y2)
 

In [ ]:
def adjustLabels(lll, labelPairs, simMatrices, xyLabs):
    numTS = len(labelPairs)
    print(numTS)
    newDict =  [copy.deepcopy(d) for d in labelPairs]
    print(newDict)
    
    ### Adjust first time-point 
    xs = [e[0] for e in xyLabs]
    ys=[e[1] for e in xyLabs]
    #labelPairs = [copy.deepcopy(d) for d in newDict]
    print(labelPairs)

    ### Now iterate
    for i in range(numTS): #for maching between slice t and t+1
        print("")
        print(f"On Slice {i} working on slice {i+1}")
        canvas=simMatrices[i]
        x = xs[i]
        y = ys[i]
        #print(canvas)
        print(x)
        print(y)
        print("")
            
        for key,val in labelPairs[i].items():
            if len(val)>=2:
                if key not in val:
                    simScores = canvas[y.index(key),:]
                    valToReplace = x[np.argmin(simScores)]
                    if isInValues(newDict[i], valToReplace)<2:
                        print(f"replace valToReplace: {valToReplace} w key:{key}")
                        #if not isInValues(newDict[i], valToReplace):
                        print(val)
                        newDict[i][key][val.index(valToReplace)]=key
                        x = [key if el==valToReplace else el for el in x]
                        if i!=numTS-1:
                            ys[i+1] = [key if el==valToReplace else el for el in ys[i+1]]
                            newDict[i+1] = replace(key, valToReplace, newDict[i+1])
                            labelPairs[i+1] = replace(key, valToReplace, labelPairs[i+1])

                            #newDict[i+1][key] = newDict[i+1][valToReplace]
                            #del newDict[i+1][valToReplace]
                            #labelPairs[i+1][key] = labelPairs[i+1][valToReplace]
                            #del labelPairs[i+1][valToReplace]
                        lll[i+1] = [key if j==valToReplace else j for j in lll[i+1]]
                    
            elif len(val)==1 and key!=val[0]:
                ### Want to deal with circumstances where cluster goes to one cluster
                ### But they have different labels
                if isInValues(newDict[i], key)==0 and isInValues(newDict[i], val[0])==1:
                    print(f"replace val[0]: {val[0]} w key:{key}")
                    newDict[i][key]=[key]
                    x = [key if el==val[0] else el for el in x]
                    lll[i+1] = [key if v==val[0] else v for v in lll[i+1]]

                    if i!=numTS-1: #Update next slice labels
                        ys[i+1] = [key if el==val[0] else el for el in ys[i+1]]
                        newDict[i+1] = replace(key, val[0], newDict[i+1])
                        labelPairs[i+1] = replace(key, val[0], labelPairs[i+1])

                        #newDict[i+1][key] = newDict[i+1][val[0]]
                        #del newDict[i+1][val[0]]
                        #labelPairs[i+1][key] = labelPairs[i+1][val[0]]
                        #del labelPairs[i+1][val[0]]

                elif isInValues(newDict[i], key)>0 and isInValues(newDict[i], val[0])==1:
                    print(f"replace key: {key} w val[0]:{val[0]}")
                    newDict[i][val[0]] = val
                    lll[i] = [val[0] if v==key else v for v in lll[i]]
                    y = [val[0] if el==key else el for el in y]
                    del newDict[i][key]
                    
            
    return lll, newDict

def isInValues(dictObject, item):
    appearances=0
    for v in dictObject.values():
        if item in v:
            appearances+=1
    return appearances

def replace(key, toChange, dictObject):
    dictObject[key] = dictObject[toChange]
    del dictObject[toChange]
    return dictObject


In [ ]:
import copy

    
lll = [l.copy() for l in bestLabels]
labelPairs = [copy.deepcopy(d) for d in labelsMatchList]

for i in range(1):
    lll, labelPairs = adjustLabels(lll, labelPairs, simMatrices, xyLabs)

print("")
print("Original Evolution Pairs")
for d in labelsMatchList:
    print(d)
print("Final Evolution Pairs")
for d in labelPairs:
    print(d)
            
perfectLabels=lll
#print(lll)
draw.auto_tiny(tempData, bestLabels, wrap='test', dim=(1,len(tempData)), same_limit=True)
draw.auto_tiny(tempData, perfectLabels, wrap='test', dim=(1,len(tempData)), same_limit=True, legend=False)

In [ ]:
### Plot finally the whole connections



### Calc Cluster Centres
### This is then used to match main clusters before finding pairings 
def connectpoints(x,y,p1,p2):
    x1, x2 = x[p1], x[p2]
    y1, y2 = y[p1], y[p2]
    plt.plot([x1,x2],[y1,y2],'k-')

    ### labelPairs has the matchings
def scatterPoints(clusterCentres):
    col = []
    toShow = np.empty((0,2), dtype=float)
    for i,d in enumerate(clusterCentres):
        v = np.vstack(list(d.values()))
        col = col+[i for x in range(len(v))]
        toShow=np.vstack((toShow,v))
        scatter = plt.scatter(toShow[:,0], toShow[:,1], c=col, s=60, cmap='spring')
    return scatter

Y = perfectLabels
'''
clusterCentresR2 = [{} for i in range(len(Y))]
for i, (l,d) in enumerate(zip(Y, tempData)):
    for n in np.unique(l):
        dMean = np.mean(d[np.where(l==n)[0]], axis=0)
        clusterCentresR2[i][n]=list(dMean)
    print(clusterCentresR2[i])
    print('\n')
'''

clusterCentresR2 = getClusterCentres(Y, tempData)
'''
col = []
toShow = np.empty((0,2), dtype=float)
for i,d in enumerate(clusterCentresR2):
    v = np.vstack(list(d.values()))
    col = col+[i for x in range(len(v))]
    toShow=np.vstack((toShow,v))
    
scatter = plt.scatter(toShow[:,0], toShow[:,1], c=col, s=60, cmap='spring')
'''
scatter = scatterPoints(clusterCentresR2)

for i in range(len(labelPairs)):
    print(labelPairs[i])
    d1=clusterCentresR2[i]
    d2=clusterCentresR2[i+1]
    for key,val in labelPairs[i].items():
        for v in val:
            plt.arrow(d1[key][0], d1[key][1], d2[v][0]-d1[key][0], d2[v][1]-d1[key][1], length_includes_head=True,
          head_width=0.1, head_length=0.2, color='black')
    
plt.legend(handles=scatter.legend_elements()[0], 
           labels=[i for i in range(len(tempData))],
           title="Time Point")
plt.show()

In [ ]:
### Plot the same arrow scatter but this time one plot for each time point

#clusterCentresR2 contains the centres at each time point i


col = []
toShow = np.empty((0,2), dtype=float)
for i in range(len(labelPairs)):
    d1 = clusterCentresR2[i]
    d2 = clusterCentresR2[i+1]
    '''
    v1 = np.vstack(list(d1.values()))
    v2 = np.vstack(list(d2.values()))
    col = [i for x in range(len(v1))]+[i+1 for x in range(len(v2))]
    toShow=np.vstack((v1,v2))
    scatter = plt.scatter(toShow[:,0], toShow[:,1], c=col, s=60, cmap='spring')
    '''
    scatter = scatterPoints([d1,d2])
    for key,val in labelPairs[i].items():
        for v in val:
            plt.arrow(d1[key][0], d1[key][1], d2[v][0]-d1[key][0], d2[v][1]-d1[key][1], length_includes_head=True,
          head_width=0.1, head_length=0.2, color='black')
    
    plt.legend(handles=scatter.legend_elements()[0], 
            labels=[i,i+1],
            #labels=[i for i in np.unique(col)],
           title="Time Point")
    plt.show()

In [ ]:
### Calc ARI for these labels
for index, l in enumerate(Labels):
    #print(f"Labels: {tempLabels[index].shape}, 2d: {labels2D[index].shape}")
    print(f"ARI:{index} {ARI(tempLabels[index], perfectLabels[index])}")
print(f"Overall ARI: {ARI(stackedLabels, [item for sublist in perfectLabels for item in sublist])}")


In [ ]:
### Calc Cluster Centres
### This is then used to match main clusters before finding pairings 
def connectpoints(x,y,p1,p2):
    x1, x2 = x[p1], x[p2]
    y1, y2 = y[p1], y[p2]
    plt.plot([x1,x2],[y1,y2],'k-')

def getClusterCentres(labels, data):
    ### Find centres of each cluster
    clusterCentres = [{} for i in range(len(labels))]
    for i, (l,d) in enumerate(zip(labels, data)):
        for n in np.unique(l):
            dMean = np.mean(d[np.where(l==n)[0]], axis=0)
            clusterCentres[i][n]=list(dMean)
    return clusterCentres

def dataToPlot(clusterCentres):
    ### Gets all clusterCentres into big 2Darray
    ### Also saves col (color) for each time point
    col = []
    toShow = np.empty((0,2), dtype=float)
    for i,d in enumerate(clusterCentres):
        v = np.vstack(list(d.values()))
        col = col + [i]*len(v)
        toShow=np.vstack((toShow,v))  
    return col, toShow

def getHung(d1,d2):
    X=[np.vstack(list(d.values())) for d in [d1,d2]]
    indices,distances=util.hungarian(X[0],X[1])
    return indices, distances, X

def plotArrows(clusterCentres):
    for i in range(len(clusterCentres)-1):
        d1=clusterCentres[i]
        d2=clusterCentres[i+1]
        (r, c),distances,X=getHung(d1,d2)
        for j,k in zip(r,c):
            plt.arrow(X[0][j][0], X[0][j][1], X[1][k][0]-X[0][j][0], X[1][k][1]-X[0][j][1], length_includes_head=True,
              head_width=0.08, head_length=0.2, color='black')



clusterCentres = getClusterCentres(bestLabels, tempData)

col, toShow = dataToPlot(clusterCentres)
scatter = plt.scatter(toShow[:,0], toShow[:,1], c=col, s=60, cmap='spring')
plotArrows(clusterCentres)
plt.legend(handles=scatter.legend_elements()[0], 
           labels=range(len(tempData)),
           title="Time Point")
plt.show()

In [ ]:
labelsCopy = [l.copy() for l in labelsXD]
labelPairsCopy=  [copy.deepcopy(d) for d in labelPairsXD]
#print(labelPairsCopy)
labelsStacked = np.hstack(labelsCopy)
print(labelsStacked[0:20])
print(labelsStacked.shape)
uniqueLabels = np.unique(labelsStacked)
print(uniqueLabels)
for i,(new,old) in enumerate(zip(range(max(uniqueLabels)), uniqueLabels)):
    if new!=old and new<len(uniqueLabels):
        ### Replace ul with ol
        for index, label in enumerate(labelsCopy):
            labelsCopy[index] = [new if l==old else l for l in labelsCopy[index]]
            if index!=len(labelPairsCopy):
                labelPairsCopy[index] = {(new if key==old else key):val for key,val in labelPairsCopy[index].items()}
                for key, val in labelPairsCopy[index].items():
                    if old in val:
                        labelPairsCopy[index][key] = [new if v==old else v for v in val]
            #if ul in labelPairsCopy[index].keys():
                #labelPairsCopy[ol] = 
            
print(np.unique(np.hstack(labelsCopy)))

print()
print(labelsCopy[0][0:20])
print("")
for i, l in enumerate(labelPairsXD):
    print(labelPairsXD[i])
    print(labelPairsCopy[i])
    print("")